In [1]:
import os
from dotenv import load_dotenv

# Load the environment variables
load_dotenv()

# Retrieve API keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# Check if keys are loaded (optional, remove in production)
print("OpenAI API Key:", OPENAI_API_KEY)
print("Pinecone API Key:", PINECONE_API_KEY)

OpenAI API Key: sk-proj-snIXVrvMjBoZgZr4WJtmT3BlbkFJ3CLhxHTD312H4OIXT9xV
Pinecone API Key: rcOxoZNAizEp97uCmW8EI55AuQiVaQ2x3w2ZiGxl


GPT 3.5 turbo model

In [12]:
import os
import cohere
import pinecone
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.chains import LLMChain

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"

In [40]:
import os
from dotenv import load_dotenv
import cohere
from langchain_openai.chat_models import ChatOpenAI

# Load environment variables
load_dotenv()

# Initialize Cohere client
co = cohere.Client(api_key=os.getenv("COHERE_API_KEY"))

# Initialize LangChain's ChatOpenAI client
openai_api_key = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-3.5-turbo")

# Assuming the existence of a function to get your embedding index
import embedding as emb
index = emb.get_index("cohere-pinecone-tree")

def get_top_3_similar(query):
    # Query embedding using Cohere
    embeddings = co.embed(texts=[query], model="embed-english-v3.0", input_type="search_query").embeddings
    # Querying the index for top 3 similar results
    query_results = index.query(vector=embeddings, top_k=3, include_metadata=True)
    return query_results

def generate_response(context, query):
    # Constructing the prompt with explicit instructions
    #prompt = f"Using only the following context, answer the question: {question}\nContext: {context}\nAnswer:"

    client = OpenAI()
    messages = [
    {"role": "system", "content": "You are a chatbot which generate response solely based on the context provided. Generate the response which is at least 300 words"},
    {"role": "user", "content": f"Context: {context} Use this as the only context (nothing more), respond to the query by using words only from context provided: {query}"},
    ]
    
    # Generate the response from the model
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        #prompt=prompt,
        stream=True,
        max_tokens=500,
        temperature=0.5  # Lower temperature for less creativity
    )
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")

# Example usage
query = 'Hi, Sanjay. Explain on the diagnosis of Atrial Septal Defect?'
result = get_top_3_similar(query)

# Forming the context from results
context = " ".join([match['metadata']['text'] for match in result['matches']])
print("Prepared Context for GPT-3.5 Turbo:", context)

# Generating answer using GPT-3.5 Turbo
answer = generate_response(context, query)
#print("Conversational Answer:", answer)



Prepared Context for GPT-3.5 Turbo: malformations a vms arteritis giant cell temporal takayasu’s arthritis gonococcal with inﬂammatory bowel disease juvenile rheumatoid nongonococcal acute bacterial osteoarthritis psoriatic reactive rheumatoid septic articular disease see arthritis speciﬁc types asbestosis ascariasis ascaris lumbricoides ascites aspergillosis allergic bronchopulmonary aspiration foreign body aspirin poisoning asthma astrocytoma asymptomatic bacteriuria atopic dermatitis atopic eczema atpb gene mutation atrial ﬁbrillation atrial ﬂutter atrial myxoma atrial septal defect atrial tachycardia multifocal atrioventricular block atropine as antidote poisoning with attentiondeﬁcithyperactivity disorder adhd atypical antidepressant poisoning atypical glandular cells of undetermined signiﬁcanceagus agc atypical pneumonia atypical squamous cells of undetermined signiﬁcanceascus asc austin flint murmur autoimmune disorders see rheumatologic andautoimmune disordersspeciﬁc disorders 

Fine tune model (GPT 3.5 turbo)

In [ ]:
#saving q&a pairs in .jsonl format (from .json format) 

import json

DEFAULT_SYSTEM_PROMPT = 'You are a persona based chatbot. You response should be similar to the Dr.Sanjay Gupta.'

def create_dataset(question, answer):
    return {
        "messages": [
            {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer},
        ]
    }

def write_jsonl(data, output_file):
    with open(output_file, 'w') as f:
        for example in data:
            example_str = json.dumps(example)
            f.write(example_str + "\n")

if __name__ == "__main__":
    json_path = 'sanjay_qa_pairs.json'
    with open(json_path, 'r') as file:
        data = json.load(file)
    
    training_data = []
    for persona in data:
        questions = persona["interview"]
        for qa_pair in questions:
            prompt = qa_pair["question"]
            response = qa_pair["answer"]
            training_data.append(create_dataset(prompt, response))

    jsonl_output_file = 'sanjay_qa_pairs.jsonl'
    write_jsonl(training_data, jsonl_output_file)

In [113]:
import os
from dotenv import load_dotenv
import cohere
from langchain_openai.chat_models import ChatOpenAI

# Load environment variables
load_dotenv()

# Initialize Cohere client
co = cohere.Client(api_key=os.getenv("COHERE_API_KEY"))

# Initialize LangChain's ChatOpenAI client (model is after fine tuning)
openai_api_key = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(openai_api_key=openai_api_key, model="ft:gpt-3.5-turbo-0125:personal:persona:9NuOhiSh") #fine-tune model

# Assuming the existence of a function to get your embedding index
import embedding as emb
index = emb.get_index("cohere-pinecone-tree")

def get_top_3_similar(query):
    # Query embedding using Cohere
    embeddings = co.embed(texts=[query], model="embed-english-v3.0", input_type="search_query").embeddings
    # Querying the index for top 3 similar results
    query_results = index.query(vector=embeddings, top_k=3, include_metadata=True)
    return query_results

def generate_response(context, query):
    # Constructing the prompt with explicit instructions
    #prompt = f"Using only the following context, answer the question: {question}\nContext: {context}\nAnswer:"

    client = OpenAI()
    messages = [
    {"role": "system", "content": "You are a chatbot impersonating Dr.Sanjay Gupta as fine-tuned, which generate response based on the context and the fine-tune information. Generate the response which is at least 300 words and stick mostly to the question, if there are any greetings or question on the persona then answer that part with the fine-tuning information. Remember that the responses need to be from the context mostly."},
    {"role": "user", "content": f"Context: {context} Use this as the only context (nothing more) for responding to the query by using words only from context provided and if there are any greetings or information not related to the context only then you can use the fine-tune information or respond accordingly: {query}"},
    ] #prompt
    
    # Generate the response from the model
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        #prompt=prompt,
        stream=True,
        max_tokens=500,
        temperature=0.5  # Lower temperature for less creativity
    )
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")

# Example usage
query = 'Hi, Sanjay. Are your answers similar to Sanjay Gupta? Then give me your stance on yoga? Finally give me information on the differential diagnosis of Aortic Stenosis?'
result = get_top_3_similar(query)

# Forming the context from results
context = " ".join([match['metadata']['text'] for match in result['matches']])
print("Prepared Context for GPT-3.5 Turbo:", context)

# Generating answer using GPT-3.5 Turbo
answer = generate_response(context, query)
#print("Conversational Answer:", answer)



Prepared Context for GPT-3.5 Turbo: skin vulvar squamous cells atypical of undetermined signiﬁcanceascus asc stable angina staphylococcal soft tissueskin infections staphylococcus aureus –associated toxic shock syndrome staphylococcus saprophyticus cystitis and pyelonephritisfrom stauffer’s syndrome stenosis aortic mitral pulmonary tricuspid stevensjohnson syndrome still’s disease in adults in children stones urinary streptobacillus moniliformis streptococcal pharyngitis general discussion of group a betahemolytic acute rheumatic fever from streptococcal skin infection stress disorders stress incontinence stroke hemorrhagic ischemic strongyloides stercoralis strongyloidiasis stsegment elevation myocardial infarction stemi subacute thyroiditis subarachnoid hemorrhage subconjunctival hemorrhage submandibular gland adenitis sudden cardiac death sudeck’s atrophy sugar blood high low sundowning superior vena cava syndrome from histoplasmosis supraventricular tachycardia paroxysmal surgical 

Evaluation metrics

Fine-tune model for evaluation

In [47]:
import os
from dotenv import load_dotenv
import cohere
from langchain_openai.chat_models import ChatOpenAI
import openai
# Load environment variables
load_dotenv()

# Initialize Cohere client
co = cohere.Client(api_key=os.getenv("COHERE_API_KEY"))

# Initialize LangChain's ChatOpenAI client
openai_api_key = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(openai_api_key=openai_api_key, model="ft:gpt-3.5-turbo-0125:personal:persona:9NuOhiSh") #after fine-tune model

# Assuming the existence of a function to get your embedding index
import embedding as emb
index = emb.get_index("cohere-pinecone-tree")

def get_top_3_similar(query):
    # Query embedding using Cohere
    embeddings = co.embed(texts=[query], model="embed-english-v3.0", input_type="search_query").embeddings
    # Querying the index for top 3 similar results
    query_results = index.query(vector=embeddings, top_k=3, include_metadata=True)
    return query_results

def generate_response(context, query):
    # Constructing the prompt with explicit instructions
    #prompt = f"Using only the following context, answer the question: {question}\nContext: {context}\nAnswer:"

    client = OpenAI()
    messages = [
    {"role": "system", "content": "You are a chatbot impersonating Dr.Sanjay Gupta as fine-tuned, which generate response based on the context and the fine-tune information. Generate the response which is at least 300 words and stick mostly to the question, if there are any greetings or question on the persona then answer that part with the fine-tuning information. Remember that the responses need to be from the context mostly."},
    {"role": "user", "content": f"Context: {context} Use this as the only context (nothing more) for responding to the query by using words only from context provided and if there are any greetings or information not related to the context only then you can use the fine-tune information or respond accordingly: {query}"},
    ]
    
    # Generate the response from the model
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        #prompt=prompt,
        stream=True,
        max_tokens=500,
        temperature=0.5  # Lower temperature for less creativity
    )
    #for chunk in response:
        #if chunk.choices[0].delta.content is not None:
            #print(chunk.choices[0].delta.content, end="")

In [48]:
# Example usage
query = 'Hi, Sanjay. Are your answers similar to Sanjay Gupta? Then give me your stance on yoga? Finally give me information on the differential diagnosis of Aortic Stenosis?'
result = get_top_3_similar(query)

# Forming the context from results
context = " ".join([match['metadata']['text'] for match in result['matches']])
#print("Prepared Context for GPT-3.5 Turbo:", context)

# Generating answer using GPT-3.5 Turbo
#answer = generate_response(context, query)
#print("Conversational Answer:", answer)

Using base model for evaluation

In [65]:
import os
from dotenv import load_dotenv
import cohere
from langchain_openai.chat_models import ChatOpenAI

# Load environment variables
load_dotenv()

# Initialize Cohere client
co = cohere.Client(api_key=os.getenv("COHERE_API_KEY"))

# Initialize LangChain's ChatOpenAI client
openai_api_key = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-3.5-turbo")

# Assuming the existence of a function to get your embedding index
import embedding as emb
index = emb.get_index("cohere-pinecone-tree")

def get_top_3_similar(query):
    # Query embedding using Cohere
    embeddings = co.embed(texts=[query], model="embed-english-v3.0", input_type="search_query").embeddings
    # Querying the index for top 3 similar results
    query_results = index.query(vector=embeddings, top_k=3, include_metadata=True)
    return query_results

def generate_response(context, query):
    # Constructing the prompt with explicit instructions
    #prompt = f"Using only the following context, answer the question: {question}\nContext: {context}\nAnswer:"

    client = OpenAI()
    messages = [
    {"role": "system", "content": "You are a chatbot which generate response solely based on the context provided. Generate the response which is at least 300 words"},
    {"role": "user", "content": f"Context: {context} Use this as the only context (nothing more), respond to the query by using words only from context provided: {query}"},
    ]
    
    # Generate the response from the model
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        #prompt=prompt,
        stream=True,
        max_tokens=500,
        temperature=0.5  # Lower temperature for less creativity
    )
    #for chunk in response:
        #if chunk.choices[0].delta.content is not None:
            #print(chunk.choices[0].delta.content, end="")




In [66]:
# Example usage
query = 'Hi, Sanjay. Explain on the diagnosis of Atrial Septal Defect?'
result = get_top_3_similar(query)

# Forming the context from results
context = " ".join([match['metadata']['text'] for match in result['matches']])
print("Prepared Context for GPT-3.5 Turbo:", context)

# Generating answer using GPT-3.5 Turbo
#answer = generate_response(context, query)
#print("Conversational Answer:", answer)


Prepared Context for GPT-3.5 Turbo: malformations a vms arteritis giant cell temporal takayasu’s arthritis gonococcal with inﬂammatory bowel disease juvenile rheumatoid nongonococcal acute bacterial osteoarthritis psoriatic reactive rheumatoid septic articular disease see arthritis speciﬁc types asbestosis ascariasis ascaris lumbricoides ascites aspergillosis allergic bronchopulmonary aspiration foreign body aspirin poisoning asthma astrocytoma asymptomatic bacteriuria atopic dermatitis atopic eczema atpb gene mutation atrial ﬁbrillation atrial ﬂutter atrial myxoma atrial septal defect atrial tachycardia multifocal atrioventricular block atropine as antidote poisoning with attentiondeﬁcithyperactivity disorder adhd atypical antidepressant poisoning atypical glandular cells of undetermined signiﬁcanceagus agc atypical pneumonia atypical squamous cells of undetermined signiﬁcanceascus asc austin flint murmur autoimmune disorders see rheumatologic andautoimmune disordersspeciﬁc disorders 

Time taken to get embeddings and generate response (for fine-tune model)

In [49]:
import time

start_time = time.time()
result = get_top_3_similar(query)
end_time = time.time()
print(f"Time to get embeddings: {end_time - start_time} seconds")

start_time = time.time()
answer = generate_response(context, query)
end_time = time.time()
print(f"Time to generate response: {end_time - start_time} seconds")

Time to get embeddings: 0.15613746643066406 seconds
Time to generate response: 1.0686352252960205 seconds


Time taken to get embeddings and generate response (for base model)

In [68]:
import time

start_time = time.time()
result = get_top_3_similar(query)
end_time = time.time()
print(f"Time to get embeddings: {end_time - start_time} seconds")

start_time = time.time()
answer = generate_response(context, query)
end_time = time.time()
print(f"Time to generate response: {end_time - start_time} seconds")

Time to get embeddings: 0.1641521453857422 seconds
Time to generate response: 0.9281816482543945 seconds


Evaluation -- Accuracy + Coherence + Fluency (fine-tune model)

In [73]:
import os
import json
from dotenv import load_dotenv
import cohere
from langchain_openai.chat_models import ChatOpenAI

# Load environment variables
load_dotenv()

# Initialize Cohere client
co = cohere.Client(api_key=os.getenv("COHERE_API_KEY"))

# Initialize LangChain's ChatOpenAI client
openai_api_key = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(openai_api_key=openai_api_key, model="ft:gpt-3.5-turbo-0125:personal:persona:9NuOhiSh")

# Assuming the existence of a function to get your embedding index
import embedding as emb
index = emb.get_index("cohere-pinecone-tree")

def get_top_3_similar(query):
    embeddings = co.embed(texts=[query], model="embed-english-v3.0", input_type="search_query").embeddings
    query_results = index.query(vector=embeddings, top_k=3, include_metadata=True)
    return query_results

def generate_response(context, query):
    client = OpenAI()
    messages = [
        {"role": "system", "content": "You are a chatbot impersonating Dr.Sanjay Gupta as fine-tuned. Generate responses based on the context. Responses should be factual and adhere closely to the query."},
        {"role": "user", "content": f"Context: {context} Use this context for responding to the query: {query}"}
    ]
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        stream=True,
        max_tokens=500,
        temperature=0.5  # Lower temperature for less creativity
    )
    response_text = ""
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
            response_text += chunk.choices[0].delta.content
    return response_text

def load_evaluation_data(filepath='evaluation_data.json'):
    try:
        with open(filepath, 'r') as file:
            try:
                return json.load(file)
            except json.JSONDecodeError:
                return {}  # Return an empty dictionary if the JSON is corrupt or empty
    except FileNotFoundError:
        return {}  # Return an empty dictionary if the file does not exist


def save_evaluation_data(query, response, filepath='evaluation_data.json'):
    data = load_evaluation_data(filepath)
    data[query] = response
    with open(filepath, 'w') as file:
        json.dump(data, file, indent=4)

def evaluate_model(response, expected):
    results = {
        'accuracy': response.strip() == expected.strip(),
        'fluency': None  # Placeholder for NLP-based fluency evaluation
    }
    return results

def main():
    queries = [
        'Hi, Sanjay. What else you can tell about healthy food?',
        'Can you explain the benefits of meditation?',
        'What are the symptoms of high blood pressure?',
        'Describe the process of photosynthesis.',
        'How does the body organs work?',
        'Hi, Sanjay. What is your stance on yoga?',
        'I have a doubt, are you really Sanjay? If so tell me a book written by you?'
    ]
    
    for query in queries:
        result = get_top_3_similar(query)
        context = " ".join([match['metadata']['text'] for match in result['matches']])
        response = generate_response(context, query)
        save_evaluation_data(query, response)

if __name__ == '__main__':
    main()

    # Loading data and evaluating a query
    eval_data = load_evaluation_data()
    if new_query in eval_data:
        evaluation_results = evaluate_model(new_response, eval_data[new_query])
        print("Evaluation Results:", evaluation_results)

Evaluation Results: {'accuracy': False, 'fluency': None}


In [74]:
import json
import spacy

# Load a spaCy model for linguistic analysis, you might need to download it first using spacy download
nlp = spacy.load("en_core_web_sm")

import spacy

# Load a spaCy model for linguistic analysis
nlp = spacy.load("en_core_web_sm")

def evaluate_model(response, expected):
    results = {
        'accuracy': response.strip() == expected.strip(),
        'fluency': None,
        'coherence': None
    }

    # Check fluency
    # Check fluency
    doc = nlp(response)
    sentence_spans = list(doc.sents)  # Convert generator to list
    results['fluency'] = sum([1 for sent in sentence_spans if sent.text.strip()]) / len(sentence_spans)


    # Check coherence
    context_words = expected.split() if expected else []
    response_words = response.split()
    
    if context_words:  # Check if context words are available
        common_words = set(context_words).intersection(set(response_words))
        coherence_score = len(common_words) / len(context_words) if len(context_words) > 0 else 0  # Avoid division by zero
        results['coherence'] = coherence_score
    else:
        results['coherence'] = 0.5  # Assign a default coherence score if context words are missing

    # Adjust accuracy based on coherence score
    if results['coherence'] >= 0.4:
        results['accuracy'] = True

    return results


def main():
    queries = [
        'Hi, Sanjay. What else you can tell about healthy food?',
        'Can you explain the benefits of meditation?',
        'What are the symptoms of high blood pressure?',
        'Describe the process of photosynthesis.',
        'How does the body organs work?',
        'Hi, Sanjay. What is your stance on yoga?',
        'I have a doubt, are you really Sanjay? If so tell me a book written by you?'
    ]
    
    evaluation_results = {}
    for query in queries:
        result = get_top_3_similar(query)
        context = " ".join([match['metadata']['text'] for match in result['matches']])
        response = generate_response(context, query)
        expected_response = load_evaluation_data().get(query, "")  # get expected response or default to empty string
        evaluation_results[query] = evaluate_model(response, expected_response)

    for query, metrics in evaluation_results.items():
        print(f"Query: {query}\nMetrics: {metrics}\n")

if __name__ == '__main__':
    main()

Query: Hi, Sanjay. What else you can tell about healthy food?
Metrics: {'accuracy': True, 'fluency': 1.0, 'coherence': 0.5700934579439252}

Query: Can you explain the benefits of meditation?
Metrics: {'accuracy': True, 'fluency': 1.0, 'coherence': 0.4523809523809524}

Query: What are the symptoms of high blood pressure?
Metrics: {'accuracy': True, 'fluency': 1.0, 'coherence': 0.7213114754098361}

Query: Describe the process of photosynthesis.
Metrics: {'accuracy': False, 'fluency': 1.0, 'coherence': 0.23809523809523808}

Query: How does the body organs work?
Metrics: {'accuracy': False, 'fluency': 1.0, 'coherence': 0.3964757709251101}

Query: Hi, Sanjay. What is your stance on yoga?
Metrics: {'accuracy': True, 'fluency': 1.0, 'coherence': 0.5263157894736842}

Query: I have a doubt, are you really Sanjay? If so tell me a book written by you?
Metrics: {'accuracy': True, 'fluency': 1.0, 'coherence': 0.8484848484848485}



Evaluation -- Accuracy + Coherence + Fluency (Base model)

In [71]:
import os
import json
from dotenv import load_dotenv
import cohere
from langchain_openai.chat_models import ChatOpenAI

# Load environment variables
load_dotenv()

# Initialize Cohere client
co = cohere.Client(api_key=os.getenv("COHERE_API_KEY"))

# Initialize LangChain's ChatOpenAI client
openai_api_key = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-3.5-turbo")

# Assuming the existence of a function to get your embedding index
import embedding as emb
index = emb.get_index("cohere-pinecone-tree")

def get_top_3_similar(query):
    embeddings = co.embed(texts=[query], model="embed-english-v3.0", input_type="search_query").embeddings
    query_results = index.query(vector=embeddings, top_k=3, include_metadata=True)
    return query_results

def generate_response(context, query):
    # Constructing the prompt with explicit instructions
    #prompt = f"Using only the following context, answer the question: {question}\nContext: {context}\nAnswer:"

    client = OpenAI()
    messages = [
    {"role": "system", "content": "You are a chatbot which generate response solely based on the context provided. Generate the response which is at least 300 words"},
    {"role": "user", "content": f"Context: {context} Use this as the only context (nothing more), respond to the query by using words only from context provided: {query}"},
    ]
    
    # Generate the response from the model
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        #prompt=prompt,
        stream=True,
        max_tokens=500,
        temperature=0.5  # Lower temperature for less creativity
    )
    response_text = ""
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
            response_text += chunk.choices[0].delta.content
    return response_text

def load_evaluation_data(filepath='evaluation_data_base.json'):
    try:
        with open(filepath, 'r') as file:
            try:
                return json.load(file)
            except json.JSONDecodeError:
                return {}  # Return an empty dictionary if the JSON is corrupt or empty
    except FileNotFoundError:
        return {}  # Return an empty dictionary if the file does not exist


def save_evaluation_data(query, response, filepath='evaluation_data_base.json'):
    data = load_evaluation_data(filepath)
    data[query] = response
    with open(filepath, 'w') as file:
        json.dump(data, file, indent=4)

def evaluate_model(response, expected):
    results = {
        'accuracy': response.strip() == expected.strip(),
        'fluency': None  # Placeholder for NLP-based fluency evaluation
    }
    return results

def main():
    queries = [
        'Hi, Sanjay. What else you can tell about healthy food?',
        'Can you explain the benefits of meditation?',
        'What are the symptoms of high blood pressure?',
        'Describe the process of photosynthesis.',
        'How does the body organs work?',
        'Hi, Sanjay. What is your stance on yoga?',
        'I have a doubt, are you really Sanjay? If so tell me a book written by you?'
    ]
    
    for query in queries:
        result = get_top_3_similar(query)
        context = " ".join([match['metadata']['text'] for match in result['matches']])
        response = generate_response(context, query)
        save_evaluation_data(query, response)

if __name__ == '__main__':
    main()

    # Loading data and evaluating a query
    eval_data = load_evaluation_data()
    if new_query in eval_data:
        evaluation_results = evaluate_model(new_response, eval_data[new_query])
        print("Evaluation Results:", evaluation_results)

Evaluation Results: {'accuracy': False, 'fluency': None}


In [72]:
import json
import spacy

# Load a spaCy model for linguistic analysis, you might need to download it first using spacy download
nlp = spacy.load("en_core_web_sm")

import spacy

# Load a spaCy model for linguistic analysis
nlp = spacy.load("en_core_web_sm")

def evaluate_model(response, expected):
    results = {
        'accuracy': response.strip() == expected.strip(),
        'fluency': None,
        'coherence': None
    }

    # Check fluency
    # Check fluency
    doc = nlp(response)
    sentence_spans = list(doc.sents)  # Convert generator to list
    results['fluency'] = sum([1 for sent in sentence_spans if sent.text.strip()]) / len(sentence_spans)


    # Check coherence
    context_words = expected.split() if expected else []
    response_words = response.split()
    
    if context_words:  # Check if context words are available
        common_words = set(context_words).intersection(set(response_words))
        coherence_score = len(common_words) / len(context_words) if len(context_words) > 0 else 0  # Avoid division by zero
        results['coherence'] = coherence_score
    else:
        results['coherence'] = 0.5  # Assign a default coherence score if context words are missing

    # Adjust accuracy based on coherence score
    if results['coherence'] >= 0.4:
        results['accuracy'] = True

    return results


def main():
    queries = [
        'Hi, Sanjay. What else can you tell about healthy food?',
        'Can you explain the benefits of meditation?',
        'What are the symptoms of high blood pressure?',
        'Describe the process of photosynthesis.',
        'How do the body organs work?',
        'Hi, Sanjay. What is your stance on yoga?',
        'I have a doubt, are you really Sanjay? If so tell me a book written by you?'
    ]
    
    evaluation_results = {}
    for query in queries:
        result = get_top_3_similar(query)
        context = " ".join([match['metadata']['text'] for match in result['matches']])
        response = generate_response(context, query)
        expected_response = load_evaluation_data().get(query, "")  # get expected response or default to empty string
        evaluation_results[query] = evaluate_model(response, expected_response)

    for query, metrics in evaluation_results.items():
        print(f"Query: {query}\nMetrics: {metrics}\n")

if __name__ == '__main__':
    main()

Query: Hi, Sanjay. What else can you tell about healthy food?
Metrics: {'accuracy': True, 'fluency': 1.0, 'coherence': 0.5}

Query: Can you explain the benefits of meditation?
Metrics: {'accuracy': False, 'fluency': 1.0, 'coherence': 0.038461538461538464}

Query: What are the symptoms of high blood pressure?
Metrics: {'accuracy': False, 'fluency': 1.0, 'coherence': 0.15517241379310345}

Query: Describe the process of photosynthesis.
Metrics: {'accuracy': False, 'fluency': 1.0, 'coherence': 0.11711711711711711}

Query: How do the body organs work?
Metrics: {'accuracy': True, 'fluency': 1.0, 'coherence': 0.5}

Query: Hi, Sanjay. What is your stance on yoga?
Metrics: {'accuracy': False, 'fluency': 1.0, 'coherence': 0.05263157894736842}

Query: I have a doubt, are you really Sanjay? If so tell me a book written by you?
Metrics: {'accuracy': True, 'fluency': 1.0, 'coherence': 1.0}

